# Iterative phase estimation using `qdk-chemistry`

This notebook provides an example of `qdk-chemistry` functionality thorough an end-to-end workflow estimating the ground state energy of a multi-configurational quantum chemistry system.  This is one of example of a wide range of functionality in `qdk-chemistry`. Please see <https://github.com/microsoft/qdk-chemistry> for the full documentation.

# TODO - FIX ME

- [ ] Standardize iQPE parameters into a single block
- [ ] Update active space talk track
- [ ] Simplify notebook text
- [ ] Fix language usage
- [ ] Add more comments to code cells
- [ ] Adjust talk track to emphasize this is one example of many workflows and features
- [ ] Replace GitHub URL with `aka.ms` URL
- [x] Rearrange or combine SCF and localization steps
- [x] Fix AutoCAS (autoCAS) capitalization
- [x] Consolidate imports
- [x] Fix subscripts in molecular formulae
- [x] Load molecule from structure file
- [x] Change all "IQPE" to "iQPE"
- [x] Quiet logger

In [ ]:
# Load frequently used external packages
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd

# Reduce logging output for demo
from qdk_chemistry.utils import Logger
Logger.set_global_level(Logger.LogLevel.off)

## Loading the stretched N<sub>2</sub> structure

This example uses a *stretched* N<sub>2</sub> molecule, which introduces multi-reference character in the wavefunction. The structure is loaded from a [XYZ-format](https://en.wikipedia.org/wiki/XYZ_file_format) file.

In [2]:
from qdk_chemistry.data import Structure

# Stretched N2 structure at 1.27 ångström bond length
structure = Structure.from_xyz_file("stretched_n2.structure.xyz")

## Generating and optimizing the molecular orbitals

This step performs a Hartree-Fock (HF) SCF calculation to generate an approximate initial wavefunction and ground-state energy guess.

Canonical molecular orbitals from SCF calculations are often delocalized over the entire molecule. As an example, we use the MP2 natural orbital localization method in `qdk-chemistry` to generate orbitals that tend to yield more compact and chemically meaningful representations.

The resulting molecular orbitals will be used in subsequent steps for active space selection and multi-configuration calculations.

In [ ]:
from qdk_chemistry.algorithms import create
from qdk_chemistry.utils import compute_valence_space_parameters

# Perform an SCF calculation, returning the energy and wavefunction
scf_solver = create("scf_solver")
E_hf, wfn_hf = scf_solver.run(structure, charge=0, spin_multiplicity=1, basis_or_guess="cc-pvdz")

# Select the valence active space and localize the orbitals
num_val_e, num_val_o = compute_valence_space_parameters(wfn_hf, charge=0)
active_space_selector = create(
    "active_space_selector",
    "qdk_valence",
    num_active_electrons=num_val_e,
    num_active_orbitals=num_val_o
)
valence_wf = active_space_selector.run(wfn_hf)
localizer = create("orbital_localizer", "qdk_mp2_natural_orbitals")
valence_indices = valence_wf.get_orbitals().get_active_space_indices()
loc_wfn = localizer.run(valence_wf, *valence_indices)
print("Localized Orbitals:\n", loc_wfn.get_orbitals().get_summary())

## Optimizing the problem with active space selection

Active space selection focuses the quantum calculation on a subset of the electrons and orbitals in the system.

This example uses `qdk_autocas_eos`, an automated entropy-based active-space selection method to identify strongly
correlated orbitals.

In [ ]:
# Construct a Hamiltonian from the localized orbitals and compute the SCI wavefunction
hamiltonian_constructor = create("hamiltonian_constructor")
loc_orbitals = loc_wfn.get_orbitals()
loc_hamiltonian = hamiltonian_constructor.run(loc_orbitals)
num_alpha_electrons, num_beta_electrons = loc_wfn.get_active_num_electrons()
macis_mc = create(
    "multi_configuration_calculator", "macis_asci", calculate_one_rdm=True, calculate_two_rdm=True,
    )
_, wfn_sci = macis_mc.run(loc_hamiltonian, num_alpha_electrons, num_beta_electrons)

# Optimize the problem with autoCAS-EOS active space selection
autocas = create("active_space_selector", "qdk_autocas_eos")
autocas_wfn = autocas.run(wfn_sci)
indices, _ = autocas_wfn.get_orbitals().get_active_space_indices()
print(f"autoCAS-EOS selected {len(indices)} of {num_val_o} orbitals for the active space:  indices={list(indices)}")


Next, we construct the active-space Hamiltonian and compute the multi-configuration wavefunction using the selected
active space.
This step also provides a reference energy for the active space system that can be used to benchmark the iQPE result.

In [ ]:
# Calculate the wavefunction and energy with CASCI
hamiltonian_constructor = create("hamiltonian_constructor")
refined_orbitals = autocas_wfn.get_orbitals()
active_hamiltonian = hamiltonian_constructor.run(refined_orbitals)

alpha_electrons, beta_electrons = autocas_wfn.get_active_num_electrons()
mc = create("multi_configuration_calculator", "macis_cas")
e_cas, wfn_cas = mc.run(active_hamiltonian, alpha_electrons, beta_electrons)
print(f"Active space system energy: {e_cas:.3f} Hartree")

Visualizing the selected active space is an important step to ensure that the selected orbitals are chemically meaningful.

In [ ]:
from qdk.widgets import MoleculeViewer
from qdk_chemistry.utils.cubegen import generate_cubefiles_from_orbitals

active_orbitals = wfn_cas.get_orbitals()

# Generate cube files for the active orbitals
cube_data = generate_cubefiles_from_orbitals(
    orbitals=active_orbitals,
    grid_size=(40, 40, 40),
    margin=10.0,
    indices=active_orbitals.get_active_space_indices()[0],
    label_maker=lambda p: f"{'occupied' if p < 20 else 'virtual'}_{p + 1:04d}"
)

# Visualize the molecular orbitals together with the structure
MoleculeViewer(molecule_data=structure.to_xyz(), cube_data=cube_data)

## Loading the wavefunction onto a quantum computer

Now that we have calculated the multi-configuration wavefunction for the active space, we can generate a quantum circuit to prepare this state on a quantum computer.

### Identifying the dominant configurations in the wavefunction

In [ ]:
from qdk.widgets import Histogram
from qdk_chemistry.utils.wavefunction import get_top_determinants

# Plot top determinant weights from the CASCI wavefunction
print(f"Total determinants in the CASCI wavefunction:  {len(wfn_cas.get_active_determinants())}")
print("Plotting the determinants by weight.")
top_configurations = get_top_determinants(wfn_cas)
display(
    Histogram(
        bar_values={k.to_string(): np.abs(v)**2 for k, v in top_configurations.items()}, 
        items="top-10", 
        sort="high-to-low",
        )
    )

### Preparing trial state with two determinants

To run quantum phase estimation, we need to prepare an initial trial state that has a good overlap with the true ground state. Here, we construct a trial state using the two most significant determinants from the multi-configuration wavefunction, parameterized by a rotation angle to adjust their contributions.

In [ ]:
from utils import prepare_2_dets_trial_state

wfn_trial, fidelity = prepare_2_dets_trial_state(wfn_cas)
print(f"Overlap of trial state with CASCI wavefunction: {fidelity:.2%}")

configurations = get_top_determinants(wfn_trial)
display(Histogram(bar_values={k.to_string(): np.abs(v)**2 for k, v in configurations.items()}, sort="high-to-low",))

### Loading the wavefunction using general state preparation methods

In [ ]:
from qdk.openqasm import estimate
from qdk.widgets import Circuit

# Generate state preparation circuit for the sparse state using the regular isometry method (Qiskit)
state_prep = create("state_prep", "regular_isometry")
regular_isometry_circuit = state_prep.run(wfn_trial)

# Visualize the regular isometry circuit
display(Circuit(regular_isometry_circuit.get_qsharp()))

# Print logical qubit counts estimated from the circuit
df = pd.DataFrame(estimate(regular_isometry_circuit.get_qasm()).logical_counts.items(), columns=['Logical Estimate', 'Counts'])
display(df)

### Loading the wavefunction using optimized state preparation methods

In [ ]:
from qdk.openqasm import estimate
from qdk.widgets import Circuit

# Generate state preparation circuit for the sparse state via sparse isometry (GF2 + X)
state_prep = create("state_prep", "sparse_isometry_gf2x")
sparse_isometry_circuit = state_prep.run(wfn_trial)

# Print logical qubit counts estimated from the circuit
df = pd.DataFrame(estimate(sparse_isometry_circuit.get_qasm()).logical_counts.items(), columns=['Logical Estimate', 'Counts'])
display(df)

# Visualize the sparse isometry circuit, idle and classical qubits are removed
display(Circuit(sparse_isometry_circuit.get_qsharp()))

## Iterative quantum phase estimation for ground state energy

Kitaev-style iterative quantum phase estimation (iQPE) estimates an eigenphase of the time-evolution operator $U = e^{-iHt}$ using one ancilla qubit and a sequence of controlled-$U^{2^k}$ applications.
 
Each iteration measures one bit of the phase (from most-significant to least-significant) and uses phase feedback to refine the estimate. In this notebook, we apply iQPE to the **qubit-mapped molecular Hamiltonian** with the prepared **trial state** to obtain the ground-state energy from the measured phase.


### Preparing qubit Hamiltonian

First, we map the classical Hamiltonian for the active space to a qubit Hamiltonian that can be measured on a quantum computer. For this example, we use the Jordan-Wigner transformation.

In [ ]:
qubit_mapper = create("qubit_mapper", algorithm_name="qiskit", encoding="jordan-wigner")
qubit_hamiltonian = qubit_mapper.run(active_hamiltonian)
print("Qubit Hamiltonian:\n", qubit_hamiltonian.get_summary())

### Visualizing the iQPE circuit

The circuit for iQPE consists of initial trial state preparation followed by multiple controlled time-evolution operations. This cell visualizes the one iteration of the iQPE circuit.

In [ ]:
from qiskit import qasm3
from qdk.openqasm import circuit
from qdk_chemistry.algorithms import IterativePhaseEstimation

M_PRECISION = 10  # number of phase bits of precision
T_TIME = 0.1  # evolution time

state_prep_circuit = qasm3.loads(sparse_isometry_circuit.qasm)
iqpe = IterativePhaseEstimation(qubit_hamiltonian, T_TIME)

# Create an iQPE circuit for a specific iteration (e.g., the third-to-last)
iter_circuit = iqpe.create_iteration_circuit(state_prep_circuit, iteration=M_PRECISION-3, total_iterations=M_PRECISION)
circuit_qasm = qasm3.dumps(iter_circuit)
display(Circuit(circuit(circuit_qasm)))

with open("iqpe_circuit_n2.qasm", "w") as f:
    f.write(circuit_qasm)


### An example single-shot low-precision iQPE run on the QDK full state simulator

In [ ]:
from utils import run_iqpe

M_PRECISION = 6
T_TIME = 0.1
NUM_TRIALS = 1
SHOTS_PER_ITERATION = 3
SIMULATOR_SEED = 42

results = run_iqpe(
    qubit_hamiltonian,
    state_prep_circuit,
    time=T_TIME,
    precision=M_PRECISION,
    shots=SHOTS_PER_ITERATION,
    seed=SIMULATOR_SEED,
    reference_energy=e_cas - active_hamiltonian.get_core_energy(),
    trials=NUM_TRIALS,
)

estimated_energy, _ = Counter([result.resolved_energy for result in results]).most_common(1)[0]

print(f"QPE Results of {M_PRECISION} bit precision:")
print(f"Reference CASCI energy: {e_cas:.6f} Hartree")
print(f"Total energy from phase estimation: {estimated_energy + active_hamiltonian.get_core_energy():.6f} Hartree")
print(f"Energy difference with CASCI energy: {abs(estimated_energy + active_hamiltonian.get_core_energy() - e_cas):.3e} Hartree")


Since single-shot phase estimates can fluctuate, we execute the iQPE routine on the QDK full state simulator for multiple independent trials to obtain a distribution of energy estimates rather than a single point value.

In [ ]:
# Large number of trials with results previously calculated and saved
from qdk_chemistry.data import QpeResult
from utils import run_iqpe

M_PRECISION = 12
T_TIME = 0.5
NUM_TRIALS = 30
SHOTS_PER_ITERATION = 3
SIMULATOR_SEED = 42
RESULTS_DIR = "results_n2"

if not Path(RESULTS_DIR).exists():
    results = run_iqpe(
        qubit_hamiltonian,
        state_prep_circuit,
        time=T_TIME,
        precision=M_PRECISION,
        shots=SHOTS_PER_ITERATION,
        seed=SIMULATOR_SEED,
        reference_energy=e_cas - active_hamiltonian.get_core_energy(),
        trials=NUM_TRIALS,
        output_dir=RESULTS_DIR,
    )

results_loaded = []
for json_file in Path(RESULTS_DIR).glob("*qpe_result.json"):
    result = QpeResult.from_json_file(json_file)
    results_loaded.append(result)
    
# Use the majority energy as the estimate
estimated_energy, _ = Counter([result.resolved_energy for result in results_loaded]).most_common(1)[0]

print(f"QPE Results of {M_PRECISION} bit precision from {NUM_TRIALS} trials:")
print(f"Reference CASCI energy: {e_cas:.6f} Hartree")
print(f"Total energy from phase estimation: {estimated_energy + active_hamiltonian.get_core_energy():.6f} Hartree")
print(f"Energy difference with CASCI energy: {abs(estimated_energy + active_hamiltonian.get_core_energy() - e_cas):.3e} Hartree")

print("Energy difference (Hartree) distribution:")
display(
    Histogram(
        bar_values={f"{abs(k+active_hamiltonian.get_core_energy()-e_cas):.3e}": v for k, v in Counter([result.resolved_energy for result in results_loaded]).items()}, 
        sort="high-to-low"
        )
    )